# Retrieving Wikipedia articles

### Fire up GraphLab Create

In [19]:
import graphlab

### Load some text data - from wikipedia, pages on people

In [20]:
people = graphlab.SFrame('people_wiki.gl/')

Data contains:  link to wikipedia article, name of person, text of article.

In [21]:
people.head()

Columns:
	URI	str
	name	str
	text	str

Rows: 10

Data:
+-------------------------------+---------------------+
|              URI              |         name        |
+-------------------------------+---------------------+
| <http://dbpedia.org/resour... |    Digby Morrell    |
| <http://dbpedia.org/resour... |    Alfred J. Lewy   |
| <http://dbpedia.org/resour... |    Harpdog Brown    |
| <http://dbpedia.org/resour... | Franz Rottensteiner |
| <http://dbpedia.org/resour... |        G-Enka       |
| <http://dbpedia.org/resour... |    Sam Henderson    |
| <http://dbpedia.org/resour... |    Aaron LaCrate    |
| <http://dbpedia.org/resour... |   Trevor Ferguson   |
| <http://dbpedia.org/resour... |     Grant Nelson    |
| <http://dbpedia.org/resour... |     Cathy Caruth    |
+-------------------------------+---------------------+
+-------------------------------+
|              text             |
+-------------------------------+
| digby morrell born 10 octo... |
| alfred j lewy aka sandy le... |
| harpdog brown is a singer ... |
| franz rottensteiner born i... |
| henry krvits born 30 decem... |
| sam henderson born october... |
| aaron lacrate is an americ... |
| trevor ferguson aka john f... |
| grant nelson born 27 april... |
| cathy caruth born 1955 is ... |
+-------------------------------+
[10 rows x 3 columns]

# Compare top words according to word counts to TF-IDF

In [22]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])
people.head()

Columns:
	URI	str
	name	str
	text	str
	word_count	dict

Rows: 10

Data:
+-------------------------------+---------------------+
|              URI              |         name        |
+-------------------------------+---------------------+
| <http://dbpedia.org/resour... |    Digby Morrell    |
| <http://dbpedia.org/resour... |    Alfred J. Lewy   |
| <http://dbpedia.org/resour... |    Harpdog Brown    |
| <http://dbpedia.org/resour... | Franz Rottensteiner |
| <http://dbpedia.org/resour... |        G-Enka       |
| <http://dbpedia.org/resour... |    Sam Henderson    |
| <http://dbpedia.org/resour... |    Aaron LaCrate    |
| <http://dbpedia.org/resour... |   Trevor Ferguson   |
| <http://dbpedia.org/resour... |     Grant Nelson    |
| <http://dbpedia.org/resour... |     Cathy Caruth    |
+-------------------------------+---------------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| digby morrell born 10 octo... | {'since': 1L, 'carltons': ... |
| alfred j lewy aka sandy le... | {'precise': 1L, 'thomas': ... |
| harpdog brown is a singer ... | {'just': 1L, 'issued': 1L,... |
| franz rottensteiner born i... | {'all': 1L, 'bauforschung'... |
| henry krvits born 30 decem... | {'legendary': 1L, 'gangste... |
| sam henderson born october... | {'now': 1L, 'currently': 1... |
| aaron lacrate is an americ... | {'exclusive': 2L, 'produce... |
| trevor ferguson aka john f... | {'taxi': 1L, 'salon': 1L, ... |
| grant nelson born 27 april... | {'houston': 1L, 'frankie':... |
| cathy caruth born 1955 is ... | {'phenomenon': 1L, 'debora... |
+-------------------------------+-------------------------------+
[10 rows x 4 columns]

In [23]:
people['tfidf'] = graphlab.text_analytics.tf_idf(people['word_count'])

In [24]:
elton = people[people['name'] == 'Elton John']

In [25]:
elton[['word_count']].stack('word_count',new_column_name=['word','count']).sort('count',ascending=False).head(3)

Columns:
	word	str
	count	int

Rows: 3

Data:
+------+-------+
| word | count |
+------+-------+
| the  |   27  |
|  in  |   18  |
| and  |   15  |
+------+-------+
[3 rows x 2 columns]

In [26]:
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False).head(3)

Columns:
	word	str
	tfidf	float

Rows: 3

Data:
+-----------+---------------+
|    word   |     tfidf     |
+-----------+---------------+
|  furnish  |  18.38947184  |
|   elton   |  17.48232027  |
| billboard | 17.3036809575 |
+-----------+---------------+
[3 rows x 2 columns]

# Measuring distance

In [27]:
victoria = people[people['name'] == 'Victoria Beckham']

In [28]:
paul = people[people['name'] == 'Paul McCartney']

In [29]:
graphlab.distances.cosine(elton['tfidf'][0], victoria['tfidf'][0])

0.9567006376655429

In [30]:
graphlab.distances.cosine(elton['tfidf'][0], paul['tfidf'][0])

0.8250310029221779

# Building nearest neighbors models with different input features and setting the distance metric

In [31]:
knn_model_tfidf = graphlab.nearest_neighbors.create(people, features=['tfidf'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

In [32]:
knn_model_word_count = graphlab.nearest_neighbors.create(people, features=['word_count'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

### Who is the nearest neighbor to 'Elton John' using raw word counts?

In [33]:
knn_model_word_count.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 12.964ms     |

| Done         |         | 100         | 104.719ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	str
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-------------------+-------------------+------+
| query_label |  reference_label  |      distance     | rank |
+-------------+-------------------+-------------------+------+
|      0      |     Elton John    | 2.22044604925e-16 |  1   |
|      0      |   Cliff Richard   |   0.16142415259   |  2   |
|      0      |   Sandro Petrone  |   0.16822542751   |  3   |
|      0      |    Rod Stewart    |   0.168327165587  |  4   |
|      0      | Malachi O'Doherty |   0.177315545979  |  5   |
+-------------+-------------------+-------------------+------+
[5 rows x 4 columns]

### Who is the nearest neighbor to 'Elton John' using TF-IDF?

In [34]:
knn_model_tfidf.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 12.969ms     |

| Done         |         | 100         | 110.215ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	str
	distance	float
	rank	int

Rows: 5

Data:
+-------------+------------------+--------------------+------+
| query_label | reference_label  |      distance      | rank |
+-------------+------------------+--------------------+------+
|      0      |    Elton John    | -2.22044604925e-16 |  1   |
|      0      |   Rod Stewart    |   0.717219667893   |  2   |
|      0      |  George Michael  |   0.747600998969   |  3   |
|      0      | Sting (musician) |   0.747671954431   |  4   |
|      0      |   Phil Collins   |   0.75119324879    |  5   |
+-------------+------------------+--------------------+------+
[5 rows x 4 columns]

### Who is the nearest neighbor to 'Victoria Beckham' using raw word counts?

In [35]:
knn_model_word_count.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 6.985ms      |

| Done         |         | 100         | 102.146ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	str
	distance	float
	rank	int

Rows: 5

Data:
+-------------+--------------------------+--------------------+------+
| query_label |     reference_label      |      distance      | rank |
+-------------+--------------------------+--------------------+------+
|      0      |     Victoria Beckham     | -2.22044604925e-16 |  1   |
|      0      | Mary Fitzgerald (artist) |   0.207307036115   |  2   |
|      0      |      Adrienne Corri      |   0.214509782788   |  3   |
|      0      |     Beverly Jane Fry     |   0.217466468741   |  4   |
|      0      |      Raman Mundair       |   0.217695474992   |  5   |
+-------------+--------------------------+--------------------+------+
[5 rows x 4 columns]

### Who is the nearest neighbor to 'Victoria Beckham' using TF-IDF?

In [36]:
knn_model_tfidf.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.968ms     |

| Done         |         | 100         | 98.247ms     |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	str
	distance	float
	rank	int

Rows: 5

Data:
+-------------+---------------------+-------------------+------+
| query_label |   reference_label   |      distance     | rank |
+-------------+---------------------+-------------------+------+
|      0      |   Victoria Beckham  | 1.11022302463e-16 |  1   |
|      0      |    David Beckham    |   0.548169610263  |  2   |
|      0      | Stephen Dow Beckham |   0.784986706828  |  3   |
|      0      |        Mel B        |   0.809585523409  |  4   |
|      0      |    Caroline Rush    |   0.819826422919  |  5   |
+-------------+---------------------+-------------------+------+
[5 rows x 4 columns]